In [1]:
# Import necessary libraries
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, Input, Dense, Reshape, Conv2DTranspose,\
    Activation, BatchNormalization, ReLU, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
# import cv2
import glob

Init Plugin
Init Graph Optimizer
Init Kernel


In [ ]:


# Set random seeds for reproducability
np.random.seed(0)
tf.random.set_seed(0)

# Define constants
NOISE_FACTOR = 0.1

# 1. Download and load the data
def load_data():
    images = []
    for img_path in glob.glob("lfwcrop_color/faces/*.ppm"):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (64, 64))
        img = img / 255.0
        images.append(img)
    images = np.array(images)
    return images

# 2. Preprocess the data
def preprocess_data(images):
    images_noisy = images + NOISE_FACTOR * np.random.normal(loc=0.0, scale=1.0, size=images.shape) 
    images_noisy = np.clip(images_noisy, 0., 1.)
    return images_noisy

# 3. Split the data into training, validation, and test sets
def split_data(images_clean, images_noisy):
    images_clean_train, images_clean_test, images_noisy_train, images_noisy_test = train_test_split(images_clean, images_noisy, test_size=0.2, random_state=0)
    return images_clean_train, images_clean_test, images_noisy_train, images_noisy_test

# 4. Create the Denoising Autoencoder model
def create_model():
    inputs = Input(shape=(64, 64, 3))
    x = inputs

    # Encoder
    for i in range(2):
        x = Conv2D(32*2**i, 3, strides=2, padding="same")(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

    x = Flatten()(x)
    x = Dense(256)(x)

    # Decoder
    x = Dense(np.prod(shape_before_flattening))(x)
    x = Reshape(shape_before_flattening)(x)

    for i in range(2):
        x = Conv2DTranspose(64//2**i, 3, strides=2, padding="same")(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

    outputs = Conv2DTranspose(3, 3, activation='sigmoid', padding='same')(x)

    # Create the model
    model = Model(inputs, outputs)
    return model

# 5. Compile and train the model
def compile_and_train_model(model, images_clean_train, images_noisy_train, images_clean_test, images_noisy_test):
    model.compile(optimizer='adam', loss='mse')
    model.fit(images_noisy_train, images_clean_train, validation_data=(images_noisy_test,
